# Text generation with (larger) LSTM Recurrent Neural Networks

By Alex Gascón Bononad - alexgascon.93@gmail.com

## 0. Introduction

### 0.1. Introduction to the Notebook
In this notebook we're going to end what we started in the first one of this repository (#1 Text generation with (larger) LSTM Recurrent Neural Networks): we''ll follow the following tutorial: http://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/ [1] to create a LSTM RNN capable of generating text. 



## 1. Description of the problem

In the previous part we were able to achieve this, but the obtained NN was too small and the generated text wasn't understandable enough. That's why now we're going to take what we learned and tested and expand it in order to get a better functional example. 

Besides, in this case, we're going to change the book we'll use to train our network: instead of Alice in Wonderland, the book to use will be "El ingenioso hidalgo don Quijote de la Mancha", one of the most famous books of Spanish literature. We have also obtained it from [Project Gutenberg](http://www.gutenberg.org/cache/epub/2000/pg2000.txt), but you can find the version without headers and footers in the same folder of this no